In [1]:
import os

import albumentations as albu
from sklearn.model_selection import train_test_split
import pandas as pd
import pytorch_lightning as pt
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchmetrics
import wandb

import datasets
import models
import transforms

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [2]:
DATA_PATH = "/hdd/zhuldyzzhan/imagenette2-320/noisy_imagenette.csv"
BATCH_SIZE = 128
NUM_WORKERS = 12
EPOCHS = 10
EXPERIMENT_NAME = "Effnet"
LR = 0.005
AUGMENTATION = "hard"

In [3]:
def get_model(model_name):
    if model_name == "Resnet":
        return models.Resnet()
    elif model_name == "VGG":
        return models.VGG()
    elif model_name == "Effnet":
        return models.Effnet()
    else:
        return models.Densenet()
    
def get_transforms(augmentation_style):
    if augmentation_style == "spatial":
        return transforms.get_spatial_transforms()
    elif augmentation_style == "hard":
        return transforms.get_hard_transforms()
    else:
        return transforms.get_light_transforms()

In [4]:
TRAIN_TRANSFORMS = get_transforms(AUGMENTATION)
train_transforms_dict = albu.to_dict(TRAIN_TRANSFORMS)

# wandb.login(key="SECRET_KEY_HERE", relogin=True)
# wandb.init(project=f"{EXPERIMENT_NAME}-{AUGMENTATION}", config=train_transforms_dict)

/opt/conda/lib/python3.8/site-packages/albumentations/imgaug/transforms.py:384: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/opt/conda/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [5]:
class ImageneteModel(pt.LightningModule):
    def __init__(self, model, criterion, optimizer, scheduler, dataloaders):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.dataloaders =  dataloaders
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10, top_k=5)
        self.f1score = torchmetrics.F1Score(task="multiclass", num_classes=10, top_k=5)
    
    def train_dataloader(self):
        return self.dataloaders["train"]
    
    def val_dataloader(self):
        return self.dataloaders["val"]
    
    def test_dataloader(self):
        return self.dataloaders["test"]

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        x, y = batch
        pred = torch.softmax(self(x), dim=1)
        # print(pred.shape, y)
        loss = self.criterion(pred, y)
        return loss
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        pred = torch.softmax(self(x), dim=1)
        loss = self.criterion(pred, y)
        accuracy = self.accuracy(pred, y)
        f1score = self.f1score(pred, y)
        history = {"val_loss": loss, "accuracy": accuracy, "f1score": f1score}
        self.log_dict(history, on_epoch=True, on_step=False, prog_bar=True)
        return history
        
    def validation_epoch_end(self, outputs):
        avg_acc = torch.stack([x["accuracy"] for x in outputs]).mean()
        self.log("avg_val_accuracy", avg_acc)
        
        # wandb.log({"avg_val_accuracy": avg_acc})
        
    def test_step(self, batch, batch_nb):
        x, y = batch
        pred = torch.softmax(self(x), dim=1)
        loss = self.criterion(pred, y)
        accuracy = self.accuracy(pred, y)
        f1score = self.f1score(pred, y)
        history = {"test_loss": loss, "test_accuracy": accuracy, "test_f1score": f1score}
        self.log_dict(history, on_epoch=False, on_step=True, prog_bar=True)
        return history
        
    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x["test_accuracy"] for x in outputs]).mean()
        self.log("avg_test_accuracy", avg_acc)
        
        # wandb.log({"avg_test_accuracy": avg_acc, "num_params": num_parameters})
        
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("loss", avg_loss)

    def configure_optimizers(self):
        return [self.optimizer], [self.scheduler]

### Imagenette

In [6]:
items_df = pd.read_csv(DATA_PATH)
items = items_df.loc[items_df["is_valid"] == False].to_dict("records")
train_items, val_items = train_test_split(items, test_size=0.3, random_state=42)


train_dataset = datasets.ImagenetDataset(train_items, TRAIN_TRANSFORMS)
val_dataset = datasets.ImagenetDataset(val_items, transforms.get_valid_transforms())

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

test_items = items_df.loc[items_df["is_valid"] == True].to_dict("records")
test_dataset = datasets.ImagenetDataset(test_items, transforms.get_valid_transforms())
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

dataLoaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

In [7]:
def train(model, dataLoaders):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)

    criterion = nn.CrossEntropyLoss()
    
    logger = TensorBoardLogger("logs/"+EXPERIMENT_NAME, name=EXPERIMENT_NAME)

    learner = ImageneteModel(model, criterion, optimizer, scheduler, dataLoaders)
    # Initialize a trainer
    trainer = pt.Trainer(
        accelerator="gpu",
        max_epochs=EPOCHS,
        precision=16,
        logger=logger,
        num_sanity_val_steps=0
    )

    # Train the model ⚡
    trainer.fit(learner)
    trainer.test(learner)

In [8]:
model = get_model(EXPERIMENT_NAME)

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:207: UserWarning: The parameter 'pretrained' is deprecated, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:220: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
train(model, dataLoaders)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | Effnet             | 4.0 M 
1 | criterion | CrossEntropyLoss   | 0     
2 | accuracy  | MulticlassAccuracy | 0     
3 | f1score   | MulticlassF1Score  | 0     
-------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
8.041     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 75/75 [00:29<00:00,  2.53it/s, loss=1.99, v_num=1, val_loss=2.090, accuracy=0.700, f1score=0.233]

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


Epoch 9: 100%|██████████| 75/75 [00:27<00:00,  2.68it/s, loss=1.91, v_num=1, val_loss=1.870, accuracy=0.853, f1score=0.284]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 75/75 [00:28<00:00,  2.66it/s, loss=1.91, v_num=1, val_loss=1.870, accuracy=0.853, f1score=0.284]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:03<00:00,  8.79it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    avg_test_accuracy       0.8560869097709656
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
from collections import OrderedDict
def load_pytorch_model(state_dict, *args, **kwargs):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if name.startswith('model.'):
            name = name.replace('model.', '') # remove `model.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    return model


In [11]:
ckpt_path = "/workspace/notebooks/logs/Effnet/Effnet/version_0/checkpoints/epoch=9-step=520.ckpt"
checkpoint = torch.load(ckpt_path)
imagenete_model = load_pytorch_model(checkpoint["state_dict"])

In [12]:
train_dataset = datasets.Cifar10SearchDataset(transform=TRAIN_TRANSFORMS)
val_dataset = datasets.Cifar10SearchDataset(train=False, transform=transforms.get_valid_transforms())

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

test_items = items_df.loc[items_df["is_valid"] == True].to_dict("records")
test_dataset = datasets.ImagenetDataset(test_items, transforms.get_valid_transforms())
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

dataLoaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

Files already downloaded and verified
Files already downloaded and verified


In [13]:
import copy

In [14]:
imagenete_last_layer = copy.deepcopy(imagenete_model.base.classifier[1])

In [15]:
train(imagenete_model, dataLoaders)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | Effnet             | 4.0 M 
1 | criterion | CrossEntropyLoss   | 0     
2 | accuracy  | MulticlassAccuracy | 0     
3 | f1score   | MulticlassF1Score  | 0     
-------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
8.041     Total estimated model params size (MB)


Epoch 0:   2%|▏         | 10/470 [00:09<06:55,  1.11it/s, loss=2.33, v_num=2]